In [1]:
import sys
import datetime as dt
sys.path.insert(0,'../..')
import matplotlib.pyplot as plt
from rivapy.instruments import ZeroCouponBondSpecification, FixedRateBond, PlainVanillaCouponBond
from rivapy.instruments.factory import create
from rivapy.tools.enums import DayCounterType, RollConvention, SecuritizationLevel, Currency

from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

%load_ext autoreload
%autoreload 2

%matplotlib inline

C:\Users\doeltz\development\RiVaPy\notebooks\instruments\../..\rivapy\__init__.py:8: UserWarning: The pyvacon module is not available. You may not use all functionality without this module. Consider installing pyvacon.
  warnings.warn('The pyvacon module is not available. You may not use all functionality without this module. Consider installing pyvacon.')


In [2]:
from collections.abc import Callable
import datetime as dt
import pandas as pd
import rivapy.tools.interfaces as interfaces
from rivapy.instruments.factory import create as _instrument_create
from rivapy.marketdata.factory import create as _marketdata_create

class MemoryStorage:
    def __init__(self, create: Callable[[dict], object]):
        self._store = {}
        self.create = create
        
    def add(self, instrument: interfaces.FactoryObject):
        if instrument.obj_id in self._store.keys():
            tmp = self._store[instrument.obj_id][-1]
            ins_dict = instrument.to_dict()
            if interfaces.FactoryObject.hash_for_dict(tmp) != interfaces.FactoryObject.hash_for_dict(ins_dict):
                self._store[instrument.obj_id].append(ins_dict)
        else:
            self._store[instrument.obj_id] = [instrument.to_dict()]
            
    def get_by_id(self, obj_id: str):
        if obj_id in self._store.keys():
            return self.create(self._store[obj_id][-1])
        else:
            raise Exception('No instrument with id ' + obj_id + ' exists in storage.')
        
    def _append_values(self, results: dict, keys: list):
        for k,v in self._store.items():
            obj = v[-1]
            for r in results.keys():
                if r == 'num_version':
                    results[r].append(len(v))
                else:
                    results[r].append(obj.get(r))
        
    def get_object_list(self, keys=['obj_id', 'cls', 'expiry', 'issue_date' ]):
        num_version = []
        tmp ={k:[] for k in keys}
        tmp['num_version'] = []
        self._append_values(tmp, keys)
        return pd.DataFrame(tmp)

In [14]:
ins_store = MemoryStorage(_instrument_create)
mkt_store = MemoryStorage(_marketdata_create)

In [29]:
for days in range (30,90, 30):
    bond = ZeroCouponBondSpecification('BOND_'+str(days), issue_date = dt.datetime(2023,1,1), maturity_date=dt.datetime(2023,1,1) + dt.timedelta(days=days), 
                              currency=Currency.EUR, notional=10.0, issuer='Depp2', 
                            securitization_level=SecuritizationLevel.SUBORDINATED)
    ins_store.add(bond)
    
ins_store.add(PlainVanillaCouponBond('PV_BOND_'+str(days), 
                                     issue_date = dt.datetime(2023,1,1),
                                     maturity_date=dt.datetime(2025,1,2), 
                                    currency=Currency.EUR, notional=1.0, issuer='Depp', 
                                    securitization_level=SecuritizationLevel.SUBORDINATED, 
                                    coupon = 0.05, coupon_freq='Y', first_coupondate = dt.datetime(2023,2,1)))

In [30]:
pv_bond = ins_store.get_by_id('PV_BOND_60')

In [32]:
pv_bond.expected_cashflows()
from datetime import timedelta
timedelta(years=1)

TypeError: 'years' is an invalid keyword argument for __new__()

In [20]:
ins_store.get_object_list(keys=['obj_id', 'cls', 'maturity_date', 'issue_date', 'issuer', 'securitisation_level' ])

,obj_id,cls,maturity_date,issue_date,issuer,securitisation_level,num_version
0,BOND_30,ZeroCouponBondSpecification,2023-01-31,2023-01-01,Depp2,None,1
1,BOND_60,ZeroCouponBondSpecification,2023-03-02,2023-01-01,Depp2,None,1
2,PV_BOND_60,PlainVanillaCouponBond,2025-01-02,2023-01-01,Depp,None,2


In [7]:
from rivapy.marketdata.curves import NelsonSiegel, DiscountCurveParametrized

ns = NelsonSiegel(beta0=0.05, beta1 = 0.1, beta2=0.1, tau=1.0)
dc = DiscountCurveParametrized('DC',  refdate = dt.datetime(2023,1,1), rate_parametrization=ns, 
                               daycounter = DayCounterType.Act365Fixed)
mkt_store.add(dc)

In [8]:
mkt_store.get_by_id('DC')
dates = [dt.datetime(2023,1,1) + dt.timedelta(days=30*days) for days in range(30)]
[dc.value(refdate = dt.datetime(2023,1,1),d=d) for d in dates]

[0.14999999983334436,
 0.14989192352864789,
 0.1495848897631583,
 0.1491028504118529,
 0.148467474410205,
 0.14769834878999621,
 0.14681316252613708,
 0.14582787477386266,
 0.1447568687932631,
 0.143613092751767,
 0.1424081884974984,
 0.14115260930667128,
 0.13985572752573716,
 0.13852593295326338,
 0.13717072273695116,
 0.1357967834973068,
 0.13441006633079175,
 0.13301585529138243,
 0.13161882989997564,
 0.1302231221856234,
 0.12883236872085,
 0.12744975807498168,
 0.1260780740742416,
 0.12471973522506187,
 0.1233768306274171,
 0.1220511526777656,
 0.12074422683620559,
 0.11945733870952498,
 0.11819155868078277,
 0.11694776429675176]